# **Stage1: Avatar gen Setup (SadTalker)**

In [ ]:
# necessary packages
!pip install numpy scipy pillow opencv-python-headless scikit-image
!pip install gdown paramiko
!pip install kornia
!pip install yacs
!pip install gfpgan
!pip install pydub
!pip install facexlib
!pip install torchaudio==2.0.0
# Install compatible versions of torch and torchvision
!pip install torch==2.0.0+cu117 torchvision==0.15.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html
# Download and install ffmpeg
!apt-get update
!apt-get install ffmpeg

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 up

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 2: Check if the SadTalker repository is present and updated
import os
if not os.path.exists(f'{base_dir}/.git'):
    # If the directory doesn't contain a git repository, clone it
    %cd /content/drive/MyDrive/
    !git clone https://github.com/OpenTalker/SadTalker.git
else:
    # If it exists, ensure it's up to date
    %cd {base_dir}
    !git pull

NameError: name 'base_dir' is not defined

In [ ]:
# Navigate to the SadTalker directory
%cd /content/drive/MyDrive/SadTalker

/content/drive/MyDrive/SadTalker


In [ ]:
# downloading the checkpoints to Google Drive
!gdown https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar -O {checkpoints_dir}/mapping_00109-model.pth.tar
!gdown https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00229-model.pth.tar -O {checkpoints_dir}/mapping_00229-model.pth.tar
!gdown https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_256.safetensors -O {checkpoints_dir}/SadTalker_V0.0.2_256.safetensors
!gdown https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_512.safetensors -O {checkpoints_dir}/SadTalker_V0.0.2_512.safetensors

# downloading facexlib and GFPGAN weights to Google Drive
!gdown https://github.com/xinntao/facexlib/releases/download/v0.1.0/alignment_WFLW_4HG.pth -O {weights_dir}/alignment_WFLW_4HG.pth
!gdown https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth -O {weights_dir}/detection_Resnet50_Final.pth
!gdown https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -O {weights_dir}/GFPGANv1.4.pth
!gdown https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth -O {weights_dir}/parsing_parsenet.pth
!gdown https://github.com/Winfredy/SadTalker/releases/download/v0.0.2/epoch_20.pth -O {checkpoints_dir}/epoch_20.pth

In [ ]:
# to download the checkpoints to Google Drive if they don't already exist
import os.path
def download_if_not_exists(url, dest):
    if not os.path.exists(dest):
        !gdown {url} -O {dest}

download_if_not_exists('https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar', f'{checkpoints_dir}/mapping_00109-model.pth.tar')
download_if_not_exists('https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00229-model.pth.tar', f'{checkpoints_dir}/mapping_00229-model.pth.tar')
download_if_not_exists('https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_256.safetensors', f'{checkpoints_dir}/SadTalker_V0.0.2_256.safetensors')
download_if_not_exists('https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_512.safetensors', f'{checkpoints_dir}/SadTalker_V0.0.2_512.safetensors')

download_if_not_exists('https://github.com/xinntao/facexlib/releases/download/v0.1.0/alignment_WFLW_4HG.pth', f'{weights_dir}/alignment_WFLW_4HG.pth')
download_if_not_exists('https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth', f'{weights_dir}/detection_Resnet50_Final.pth')
download_if_not_exists('https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth', f'{weights_dir}/GFPGANv1.4.pth')
download_if_not_exists('https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth', f'{weights_dir}/parsing_parsenet.pth')
download_if_not_exists('https://github.com/Winfredy/SadTalker/releases/download/v0.0.2/epoch_20.pth', f'{checkpoints_dir}/epoch_20.pth')

# **Stage2: Text generation Setup (GPT3.5)**

In [ ]:
!pip install pandas numpy transformers langchain sentence-transformers chromadb
# installing the required packages
!pip install langchain langchain_community langgraph gtts
!pip install gpt4all
!pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#librarries
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from gtts import gTTS
from IPython.display import display, HTML, Video
import os
import moviepy.editor as mp
from urllib.parse import urlparse
from diffusers import StableDiffusionPipeline
import torch

# **Stage 3: Text generation implmentation using GPT-3.5**

In [ ]:
# LLM pipeline using a local model
model_name = "EleutherAI/gpt-neo-2.7B"  # Example model, you can choose another
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

# loading the data
loader = CSVLoader(file_path="/content/drive/MyDrive/stage1_all.csv")
data = loader.load()

# data transformers
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=0)
texts = text_splitter.split_documents(data)

#  vectordB
vectorstore = Chroma.from_documents(documents=texts, collection_name="rag-chroma", embedding=GPT4AllEmbeddings())

# retriever
retriever = vectorstore.as_retriever()

# defining the format_docs function
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Ddfining the prompt for the RAG chain
qa_prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise.

    Question: {question}
    Context: {context}
    Answer: """,
    input_variables=["question", "context"],
)

#retrieve function
def retrieve(question):
    print("---RETRIEVE---")
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

#the generate function
def generate(state):
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    context = format_docs(documents)
    prompt_str = qa_prompt.format(question=question, context=context)

    response = llm(prompt_str, max_new_tokens=200, truncation=True, do_sample=True)
    generation = response[0]['generated_text'].strip().split('Answer: ')[-1].strip()
    return {"documents": documents, "question": question, "generation": generation}

#text to speech function
def text_to_speech(text, filename="output.mp3"):
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    print(f"Audio file saved as {filename}")

# workflow execution function
def execute_workflow(question):
    state = retrieve(question)
    state = generate(state)
    text_to_speech(state["generation"], filename="generated_audio.mp3")
    return state["generation"]

# Compile the workflow
app = execute_workflow

# Test
inputs = {"question": "give me a list the places to visit Chicago with my partner"}
generation = app(inputs["question"])
if generation:
    print(f"Generated Answer: {generation}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---RETRIEVE---
---GENERATE---
Audio file saved as generated_audio.mp3
Generated Answer: The beautiful and historic Old Town South of Chicago is a great city for shopping, for example, for the home furnishings, a variety of shops, the Art Deco architecture and the shopping possibilities.

    Sentence fragment:  
    A Chicagoan knows that the city has a rich history,

and it had become one of the largest and most important cities on the east coast.
    Answer:


# **Stage 4: Text to audio conversion implmentation using google gTTS**

In [ ]:
from gtts import gTTS
import os

# Convert text to audio
def text_to_audio(text, filename):
    tts = gTTS(text, lang='en')
    tts.save(filename)

audio_filename = "/content/drive/MyDrive/SadTalker/examples/driven_audio/generated_audio.wav"
text_to_audio(generation, audio_filename)
print(f"Audio file saved as {audio_filename}")

Audio file saved as /content/drive/MyDrive/SadTalker/examples/driven_audio/generated_audio.wav


# **Stage 5: Avatar video generation using SadTalker**

In [ ]:
# Run SadTalker inference
source_image_path = "/content/drive/MyDrive/SadTalker/examples/source_image/michelle.png"
result_dir = "/content/drive/MyDrive/SadTalker/results"
checkpoint_dir = "/content/drive/MyDrive/SadTalker/checkpoints"
video_output_path = os.path.join(result_dir, "final_output.mp4")

In [ ]:
!python /content/drive/MyDrive/SadTalker/inference.py --driven_audio {audio_filename} --source_image {source_image_path} --result_dir {result_dir} --still --preprocess full --enhancer gfpgan --checkpoint_dir {checkpoint_dir} --batch_size 1

# Get the latest generated video
def get_latest_generated_video(result_dir):
    video_files = [os.path.join(result_dir, file) for file in os.listdir(result_dir) if file.endswith(".mp4")]
    if not video_files:
        return None
    latest_video = max(video_files, key=os.path.getmtime)
    return latest_video

generated_video = get_latest_generated_video(result_dir)
print(f"Generated Video: {generated_video}")

using safetensor as default
3DMM Extraction for source image
landmark Det:: 100% 1/1 [00:00<00:00,  9.93it/s]
3DMM Extraction In Video:: 100% 1/1 [00:00<00:00, 17.24it/s]
mel:: 100% 647/647 [00:00<00:00, 40720.18it/s]
audio2exp:: 100% 65/65 [00:00<00:00, 326.57it/s]
Face Renderer:: 100% 647/647 [00:20<00:00, 31.09it/s]
The generated video is named /content/drive/MyDrive/SadTalker/results/2024_05_16_01.34.42/michelle##generated_audio.mp4
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
seamlessClone:: 100% 647/647 [00:05<00:00, 109.52it/s]
The generated video is named /content/drive/MyDrive/SadTalker/results/2024_05_16_01.34.42/michelle##generated_audio_full.mp4
face enhancer....
Face Enhancer:: 100% 647/647 [01:34<00:00,  6.84it/s]
The generated video is named /content/drive/MyDrive/SadTalker/results/2024_05_16_01.34.42/michelle##generated_audio_enhanced.mp4
The generate

In [ ]:
# Display the video in the notebook
if generated_video:
    display(Video(generated_video, embed=True, width=600, height=400))
else:
    print("No video file generated.")